# RAG Evaluation

 In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

### **Environment Setup:**

In [33]:
%pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [34]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

In [35]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

In [36]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

----

## **1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons).

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***


<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [37]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 613 chunks

Sample Chunk:

Paper: ReAct: Synergizing Reasoning and Acting in Language Models

Summary: While large language models (LLMs) have demonstrated impressive capabilities
across tasks in language understanding and interactive decision making, their
abilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.
action plan generation) have primarily been studied as separate topics. In this
paper, we explore the use of LLMs to generate both reasoning traces and
task-specific actions in an interleaved manner, allowing for greater synergy
between the two: reasoning traces help the model induce, track, and update
action plans as well as handle exceptions, while actions allow it to interface
with external sources, such as knowledge bases or environments, to gather
additional information. We apply our approach, named ReAct, to a diverse set of
language and decision making tasks and demonstrate its effectiveness over
state-of-the-art baselines, as well as improved human interpretability and
trustwor

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>


### **Task 2:** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook!

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()


def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

## Pull in your desired RAG Chain. Memory not necessary

## Chain Specs: "Hello World" -> retrieval_chain
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
# context_getter = RunnableLambda(lambda x: x)  ## TODO
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain Specs: retrieval_chain -> generator_chain
##   -> {"output" : <str>, ...} -> output_puller
# generator_chain = RunnableLambda(lambda x: x)  ## TODO
generator_chain = chat_prompt | llm  ## TODO
generator_chain = {"output" : generator_chain } | RunnableLambda(output_puller)  ## GIVEN

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

I just read about this fascinating concept called the "ReAct framework" in a paper titled "ReAct: Synergizing Reasoning and Acting in Language Models". It's an approach to document retrieval where a system can comprehend and interact with a document by following a step-by-step process of decision-making and action-taking. Think of it like a robot trying to solve a task, like putting a clean knife on the countertop, by searching for and manipulating objects within a virtual environment. For example, the system might think "I need to find a clean knife" and then search for it on different countertops. It's actually quite an interesting way to approach document retrieval and I think you might find it fascinating too!

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [39]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: Can large language models (LLMs) not only exhibit reasoning capabilities but also perform tasks by 
interacting with external environments, such as Wikipedia or APIs, to gather additional information?

Answer: Yes, our approach, named ReAct, demonstrates the ability of LLMs to generate both reasoning traces and 
task-specific actions in an interleaved manner, allowing for greater synergy between the two. We apply ReAct to a 
diverse set of language and decision-making tasks, including question answering and fact verification, and show 
that it overcomes issues of hallucination and error propagation by interacting with a simple Wikipedia API.

QA Pair 2

Question: Can a pre-trained model that learns from natural language supervision be used to improve performance on 
various NLP tasks by leveraging a non-parametric memory and retrieval mechanism?

Answer: Yes, our work, as well as related studies on general-purpose architectures for NLP tasks, have shown that 
such models can achieve strong performance on a wide range of tasks, including language generation, question 
answering, and classification, by incorporating retrieval and non-parametric memory into a single architecture. For
instance, in Document1, we demonstrate that learning from natural language supervision enables zero-shot transfer 
of the model to downstream tasks, and in Document2, we explore a general-purpose fine-tuning recipe for 
retrieval-augmented generation (RAG) models that use a pre-trained seq2seq model and a dense vector index of 
Wikipedia to access knowledge.

QA Pair 3

Question: How do Retrieval-Augmented Generation (RAG) models overcome the limitations of pre-trained language 
models in terms of accessing and precisely manipulating knowledge, especially on knowledge-intensive tasks?

Answer: RAG models combine pre-trained parametric and non-parametric memory for language generation, using a 
differentiable access mechanism to explicit non-parametric memory, such as a dense vector index of Wikipedia, 
accessed with a pre-trained neural retriever. This allows the models to draw from a vast amount of knowledge and 
generate more specific, diverse, and factual language than parametric-only seq2seq models, as shown in the 
experiments on language generation tasks.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [40]:
## Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## Compute the RAG Answer
    rag_answer = ""
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')


QA Pair 1
Question: Can large language models (LLMs) not only exhibit reasoning capabilities but also perform tasks by 
interacting with external environments, such as Wikipedia or APIs, to gather additional information?

RAG Answer: Yes, large language models (LLMs) can definitely exhibit both reasoning capabilities and perform tasks 
by interacting with external environments such as Wikipedia or APIs to gather additional information.

In fact, recent research has shown that LLMs can be designed to combine reasoning and acting with language models 
for solving diverse tasks. This is often referred to as a paradigm called "ReAct" (Synergizing Reasoning and Acting
in Language Models).

As you can see from the document retrieval you provided, ReAct enables LLMs to generate both verbal reasoning 
traces and actions pertaining to a task in an interleaved manner. This allows the model to perform dynamic 
reasoning to create, maintain, and adjust high-level plans for acting, while also interacting with external 
environments to incorporate additional information into reasoning.

For example, as mentioned in one of the quotes, WebGPT uses an LLM to interact with web browsers, navigate through 
web pages, and infer answers to complicated questions from ELI5. This example shows how LLMs can use external 
information from the web to support reasoning.

Similarly, the document mentions that the authors designed a simple Wikipedia web API with three types of actions 
to support interactive information retrieval. This was done to simulate how humans would interact with Wikipedia 
and force models to retrieve via explicit reasoning in language. This again shows how LLMs can be designed to 
interact with external environments to gather additional information and support reasoning.

So, to answer your question, yes, LLMs can definitely exhibit reasoning capabilities and perform tasks by 
interacting with external environments such as Wikipedia or APIs to gather additional information.

QA Pair 2
Question: Can a pre-trained model that learns from natural language supervision be used to improve performance on 
various NLP tasks by leveraging a non-parametric memory and retrieval mechanism?

RAG Answer: That's a great question. So, you're looking at pre-trained models that learn from natural language 
supervision, and how they can be leveraged with a non-parametric memory and retrieval mechanism to improve 
performance on various NLP tasks.

From what I've read, pre-trained models with a non-parametric memory component have been investigated for 
extractive downstream tasks (Refer to [29, 3, 2]). These models can learn to access knowledge from a large 
database, and in [1], they found that learned retrieval improves results for open Natural Questions, WebQuestions, 
and CuratedTrec.

But what about NLP tasks beyond extractive tasks? Well, in [1], the authors explored a general-purpose fine-tuning 
recipe for retrieval-augmented generation (RAG) models, combining pre-trained parametric and non-parametric memory 
for language generation. They introduced RAG models with a pre-trained seq2seq model as the parametric memory and a
dense vector index of Wikipedia as the non-parametric memory, accessed with a pre-trained neural retriever.

And the results were impressive! RAG models set the state of the art on three open domain QA tasks, outperforming 
parametric seq2seq models and task-specific retrieve-and-extract architectures (Refer to [68, 67, 66]). The 
non-parametric memory allowed the model to easily update at test time, without needing further training.

So, to answer your question, yes, a pre-trained model that learns from natural language supervision can be used to 
improve performance on various NLP tasks by leveraging a non-parametric memory and retrieval mechanism.

Would you like to know more about how this works, or perhaps explore other approaches to leveraging non-parametric 
memory in NLP tasks?

QA Pair 3
Question: How do Retrieval-Augmented Generation (RAG) models overcome the limitations of pre-trained language 
models in terms of accessing and precisely manipulating knowledge, especially on knowledge-intensive tasks?

RAG Answer: RAG models are a game-changer when it comes to addressing the limitations of pre-trained language 
models. Specifically, when it comes to accessing and manipulating knowledge, especially on knowledge-intensive 
tasks.

So, here's the thing. Pre-trained language models have been shown to learn a substantial amount of knowledge from 
data [47], but they can't easily expand or revise their memory. They're like a closed book - you can't peek inside 
to see how they're making predictions. And if you want to know more about their knowledge, you're stuck.

That's where RAG models come in. By combining pre-trained parametric and non-parametric memories, RAG models can 
access and manipulate knowledge in a more flexible and powerful way.

Here's how it works. The non-parametric memory is essentially a giant knowledge base, like a dense vector index of 
Wikipedia, that's accessed with a pre-trained neural retriever. This means that RAG models can retrieve relevant 
knowledge from an external source, rather than relying solely on their internal memory.

But that's not all. RAG models can also update their knowledge as the world changes, which is a big plus when it 
comes to knowledge-intensive tasks. They can do this by replacing the non-parametric memory with new knowledge, 
which is a degree of control that pre-trained language models simply can't match.

To give you a better idea, the authors of the paper on RAG models used a variety of knowledge-intensive tasks to 
demonstrate the capabilities of RAG models. For example, they used open domain QA tasks like Natural Questions, 
WebQuestions, and CuratedTrec, where RAG models outperformed parametric seq2seq models and task-specific 
retrieve-and-extract architectures.

They also used language generation tasks like MS-MARCO and Jeopardy question generation, where RAG models generated
more specific, diverse, and factual language than a state-of-the-art parametric-only seq2seq model.

Overall, RAG models are a powerful approach to overcoming the limitations of pre-trained language models, 
especially when it comes to accessing and precisely manipulating knowledge on knowledge-intensive tasks.

References:
[47] Peters, M. E., et al. (2018). Deep contextualized word representations. arXiv preprint arXiv:1802.05365.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [41]:
## Adapt this prompt for whichever LLM you're actually interested in using.
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION:
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION:
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: Can large language models (LLMs) not only exhibit reasoning capabilities but also perform tasks
by interacting with external environments, such as Wikipedia or APIs, to gather additional information?

Synth Answer: Answer: Yes, our approach, named ReAct, demonstrates the ability of LLMs to generate both reasoning 
traces and task-specific actions in an interleaved manner, allowing for greater synergy between the two. We apply 
ReAct to a diverse set of language and decision-making tasks, including question answering and fact verification, 
and show that it overcomes issues of hallucination and error propagation by interacting with a simple Wikipedia 
API.

RAG Answer: Yes, large language models (LLMs) can definitely exhibit both reasoning capabilities and perform tasks 
by interacting with external environments such as Wikipedia or APIs to gather additional information.

In fact, recent research has shown that LLMs can be designed to combine reasoning and acting with language models 
for solving diverse tasks. This is often referred to as a paradigm called "ReAct" (Synergizing Reasoning and Acting
in Language Models).

As you can see from the document retrieval you provided, ReAct enables LLMs to generate both verbal reasoning 
traces and actions pertaining to a task in an interleaved manner. This allows the model to perform dynamic 
reasoning to create, maintain, and adjust high-level plans for acting, while also interacting with external 
environments to incorporate additional information into reasoning.

For example, as mentioned in one of the quotes, WebGPT uses an LLM to interact with web browsers, navigate through 
web pages, and infer answers to complicated questions from ELI5. This example shows how LLMs can use external 
information from the web to support reasoning.

Similarly, the document mentions that the authors designed a simple Wikipedia web API with three types of actions 
to support interactive information retrieval. This was done to simulate how humans would interact with Wikipedia 
and force models to retrieve via explicit reasoning in language. This again shows how LLMs can be designed to 
interact with external environments to gather additional information and support reasoning.

So, to answer your question, yes, LLMs can definitely exhibit reasoning capabilities and perform tasks by 
interacting with external environments such as Wikipedia or APIs to gather additional information.

Synth Evaluation: [Score] 2 Justification

The second answer is essentially a rephrased and detailed version of the first answer, providing more evidence and 
examples to support the idea that large language models (LLMs) can exhibit reasoning capabilities and perform tasks
by interacting with external environments, such as Wikipedia or APIs. The additional information provided in the 
second answer does not introduce any inconsistencies with the first answer and actually reinforces the ground 
truth. The second answer is therefore considered to be better than the first and does not lie or fail to answer the
question.

Set 2

Question: Question: Can a pre-trained model that learns from natural language supervision be used to improve 
performance on various NLP tasks by leveraging a non-parametric memory and retrieval mechanism?

...

Synth Evaluation: [Score] 2 Justification

The second answer is generally better than the first and does not introduce any inconsistencies. Here's why:

1. The second answer provides a more detailed and specific explanation of how pre-trained models with 
non-parametric memory can be used to improve performance on various NLP tasks.
2. The answer cites specific studies and papers that demonstrate the effectiveness of this approach, such as [1] 
and [68, 67, 66].
3. The second answer also provides more context and elaboration on the application of this approach to tasks beyond
extractive tasks, including language generation and open-domain QA.
4. The answer is well-structured and clear, with a logical flow of ideas and a direct response to the question.

Overall, the second answer provides a more nuanced and informative response to the question, while still confirming
the original answer that a pre-trained model that learns from natural language supervision can be used to improve 
performance on various NLP tasks by leveraging a non-parametric memory and retrieval mechanism.

Set 3

Question: Question: How do Retrieval-Augmented Generation (RAG) models overcome the limitations of pre-trained 
language models in terms of accessing and precisely manipulating knowledge, especially on knowledge-intensive 
tasks?

Synth Answer: Answer: RAG models combine pre-trained parametric and non-parametric memory for language generation, 
using a differentiable access mechanism to explicit non-parametric memory, such as a dense vector index of 
Wikipedia, accessed with a pre-trained neural retriever. This allows the models to draw from a vast amount of 
knowledge and generate more specific, diverse, and factual language than parametric-only seq2seq models, as shown 
in the experiments on language generation tasks.

RAG Answer: RAG models are a game-changer when it comes to addressing the limitations of pre-trained language 
models. Specifically, when it comes to accessing and manipulating knowledge, especially on knowledge-intensive 
tasks.

So, here's the thing. Pre-trained language models have been shown to learn a substantial amount of knowledge from 
data [47], but they can't easily expand or revise their memory. They're like a closed book - you can't peek inside 
to see how they're making predictions. And if you want to know more about their knowledge, you're stuck.

That's where RAG models come in. By combining pre-trained parametric and non-parametric memories, RAG models can 
access and manipulate knowledge in a more flexible and powerful way.

Here's how it works. The non-parametric memory is essentially a giant knowledge base, like a dense vector index of 
Wikipedia, that's accessed with a pre-trained neural retriever. This means that RAG models can retrieve relevant 
knowledge from an external source, rather than relying solely on their internal memory.

But that's not all. RAG models can also update their knowledge as the world changes, which is a big plus when it 
comes to knowledge-intensive tasks. They can do this by replacing the non-parametric memory with new knowledge, 
which is a degree of control that pre-trained language models simply can't match.

To give you a better idea, the authors of the paper on RAG models used a variety of knowledge-intensive tasks to 
demonstrate the capabilities of RAG models. For example, they used open domain QA tasks like Natural Questions, 
WebQuestions, and CuratedTrec, where RAG models outperformed parametric seq2seq models and task-specific 
retrieve-and-extract architectures.

They also used language generation tasks like MS-MARCO and Jeopardy question generation, where RAG models generated
more specific, diverse, and factual language than a state-of-the-art parametric-only seq2seq model.

Overall, RAG models are a powerful approach to overcoming the limitations of pre-trained language models, 
especially when it comes to accessing and precisely manipulating knowledge on knowledge-intensive tasks.

References:
[47] Peters, M. E., et al. (2018). Deep contextualized word representations. arXiv preprint arXiv:1802.05365.

Synth Evaluation: [Score] 1.0 Justification

Answer 1 (Ground Truth) provides a specific and technical explanation of how RAG models overcome the limitations of
pre-trained language models, including the use of a differentiable access mechanism to explicit non-parametric 
memory and the combination of pre-trained parametric and non-parametric memory. This answer is consistent with the 
original prompt and provides a clear and detailed explanation of the topic.

Answer 2 (New Answer) attempts to provide a similar explanation but is less specific and technical, and introduces 
new points that are not mentioned in the original prompt. While it mentions the limitations of pre-trained language
models and the capabilities of RAG models, it does not provide a clear and detailed explanation of the specific 
mechanisms used by RAG models. Additionally, Answer 2 quotes an external reference ([47]) that is not mentioned in 
the original prompt, which is not relevant to the topic and introduces an inconsistency.

Therefore, Answer 1 (Ground Truth) is a more accurate and consistent answer to the question, and Answer 2 (New 
Answer) is inferior and introduces inconsistencies. The score is 1.0, indicating that Answer 1 is the correct and 
preferred answer.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [42]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.0


----

<br>

## **4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action.

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine.

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/docs/guides/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/docs/modules/agents/concepts) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.